In [1]:
import numpy as np
import os
import pandas as pd
import nltk
from nltk.corpus import stopwords
en_stops = set(stopwords.words('english'))

# Extracting the Stanford Feature Set

In [2]:
maindir = os.getcwd()
os.chdir("/../../datasets/aclImdb/")

In [3]:
voc_f = open("imdb.vocab",'r')
vocab = voc_f.read().split("\n")

In [4]:
Exp_f = open("imdbEr.txt",'r')
EV = Exp_f.read().split("\n")[:-1]

In [5]:
vocabInfo = pd.DataFrame({"vocab":vocab,"expected":EV})
vocabInfo

,vocab,expected
0,the,0.0490972013402
1,and,0.201363575849
2,a,0.0333946807184
3,of,0.099837669572
4,to,-0.0790210365788
...,...,...
89522,copywrite,0
89523,artbox,0
89524,kinky-sex,0
89525,urrrghhh,0


In [6]:
vocabInfo = vocabInfo[~ vocabInfo["vocab"].isin(en_stops)]
vocabInfo["expected"] = abs(vocabInfo["expected"].astype(float))
vocabInfo

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,vocab,expected
15,movie,0.246354
17,film,0.147886
26,one,0.087114
27,!,0.134839
36,like,0.280548
...,...,...
89522,copywrite,0.000000
89523,artbox,0.000000
89524,kinky-sex,0.000000
89525,urrrghhh,0.000000


In [7]:
vocabInfo.sort_values(by="expected",ascending=False)[:round(len(vocabInfo)*0.05)]

,vocab,expected
54867,pelswick,4.500000
33943,magorian,4.500000
67306,subspace,4.500000
35092,jgar,4.500000
58224,buh,4.500000
...,...,...
81702,danira,2.267657
61418,ecclesiastical,2.267421
40382,grey-haired,2.267421
44828,irland,2.267421


In [8]:
sub = (vocabInfo[vocabInfo["expected"] <= 2].sort_values(by="expected"))
reducedVocab = sub.sample(n=round(len(sub)*0.1),random_state=1)
reducedVocab
len(reducedVocab)

8347

# Train

In [9]:
os.chdir("train/")

In [10]:
f = open("labeledBow.feat",'r')
reviews = f.read().split("\n")
ratings = []
BOW = []
for i in reviews:
    elements = i.split(" ")
    ratings.append((elements[0]))
    BOW.append(elements[1:])
f.close()

In [11]:
BOW = [[j.split(":") for j in i] for i in BOW]

In [12]:
BOW

[[['0', '9'],
  ['1', '1'],
  ['2', '4'],
  ['3', '4'],
  ['4', '6'],
  ['5', '4'],
  ['6', '2'],
  ['7', '2'],
  ['8', '4'],
  ['10', '4'],
  ['12', '2'],
  ['26', '1'],
  ['27', '1'],
  ['28', '1'],
  ['29', '2'],
  ['32', '1'],
  ['41', '1'],
  ['45', '1'],
  ['47', '1'],
  ['50', '1'],
  ['54', '2'],
  ['57', '1'],
  ['59', '1'],
  ['63', '2'],
  ['64', '1'],
  ['66', '1'],
  ['68', '2'],
  ['70', '1'],
  ['72', '1'],
  ['78', '1'],
  ['100', '1'],
  ['106', '1'],
  ['116', '1'],
  ['122', '1'],
  ['125', '1'],
  ['136', '1'],
  ['140', '1'],
  ['142', '1'],
  ['150', '1'],
  ['167', '1'],
  ['183', '1'],
  ['201', '1'],
  ['207', '1'],
  ['208', '1'],
  ['213', '1'],
  ['217', '1'],
  ['230', '1'],
  ['255', '1'],
  ['321', '5'],
  ['343', '1'],
  ['357', '1'],
  ['370', '1'],
  ['390', '2'],
  ['468', '1'],
  ['514', '1'],
  ['571', '1'],
  ['619', '1'],
  ['671', '1'],
  ['766', '1'],
  ['877', '1'],
  ['1057', '1'],
  ['1179', '1'],
  ['1192', '1'],
  ['1402', '2'],
  ['1416', 

In [13]:
# Removing words
l=list(newVocab["vocab"])
for i in range(len(BOW)): #rev
    keep = {}
    for e in BOW[i]: #word
        if vocab[int(e[0])] in l:
            e[0] = vocab[int(e[0])]
            e[1] = int(e[1])
            keep[e[0]] = e[1]
    BOW[i] = keep
    

In [14]:
BOW = pd.Series(BOW)
BOW.head()

0    {'life': 1, 'comedy': 1, 'whole': 1, 'reality'...
1    {'good': 3, 'also': 2, 'comedy': 1, 'stories':...
2                       {'cross': 1, 'correctness': 1}
3    {'something': 1, 'experiment': 1, 'participant...
4    {'also': 1, 'love': 3, 'something': 1, 'must':...
dtype: object

In [15]:
tf = pd.DataFrame(list(BOW[:-1]))

In [16]:
tf = tf.fillna(0)
df = (tf > 0).sum(axis=0)
idf = np.log(len(tf) / df)
tf_idf_train = tf * idf
#np.save("idf.npy",idf)
# AFTER CAN JUMP TO TEST

In [17]:
ratings = np.array([int(i) for i in ratings[:-1]])
ratings = pd.Series((2*(ratings > 5).astype(int))-1)
tf_idf_train["Label"] =  ratings

In [18]:
tf_idf_train.head()

,life,comedy,whole,reality,cartoon,good,also,stories,behaved,cross,...,amazons-dr,now-over,creature-basically,mud-wrestling,multi-sexual,barabarian,gamorrean,d-grade,highly-entertaining,Label
0,1.734095,2.38033,2.250752,3.455865,4.257334,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.000000,2.38033,0.000000,0.000000,0.000000,2.900542,2.707422,3.329807,7.082109,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.675593,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.353711,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [19]:
os.chdir(maindir)
os.chdir("..")
#tf_idf_train.to_csv("tf_idf_train2.csv")

In [20]:
tf_idf_train

,life,comedy,whole,reality,cartoon,good,also,stories,behaved,cross,...,amazons-dr,now-over,creature-basically,mud-wrestling,multi-sexual,barabarian,gamorrean,d-grade,highly-entertaining,Label
0,1.734095,2.38033,2.250752,3.455865,4.257334,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1
1,0.000000,2.38033,0.000000,0.000000,0.000000,2.900542,2.707422,3.329807,7.082109,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1
2,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.675593,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1
3,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1
4,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.353711,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0.000000,0.00000,0.000000,0.000000,0.000000,0.966847,1.353711,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.126631,-1
24996,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,-1
24997,0.000000,0.00000,0.000000,0.000000,0.000000,1.933695,1.353711,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,-1
24998,0.000000,0.00000,2.250752,0.000000,0.000000,2.900542,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,-1


# Test

In [21]:
os.chdir("/../../datasets/aclImdb/test")

In [22]:
f = open("labeledBow.feat",'r')
reviews = f.read().split("\n")
ratings = []
BOW = []
for i in reviews:
    elements = i.split(" ")
    ratings.append((elements[0]))
    BOW.append(elements[1:])
f.close()

In [23]:
BOW = [[j.split(":") for j in i] for i in BOW]

In [24]:
# Removing words
l=list(newVocab["vocab"])
for i in range(len(BOW)): #rev
    keep = {}
    for e in BOW[i]: #word
        if vocab[int(e[0])] in l:
            e[0] = vocab[int(e[0])]
            e[1] = int(e[1])
            keep[e[0]] = e[1]
    BOW[i] = keep
    

In [25]:
BOW = pd.Series(BOW)

In [26]:
BOW

0        {'good': 1, 'well': 2, 'comedy': 1, 'overcome'...
1                                   {'good': 1, 'well': 1}
2        {'well': 3, 'also': 1, 'love': 1, 'something':...
3        {'good': 1, 'well': 3, 'bad': 2, 'find': 1, 'c...
4                       {'highly': 1, 'under-promoted': 1}
                               ...                        
24996    {'life': 1, 'real': 2, 'written': 1, 'usual': ...
24997    {'plot': 3, 'run': 1, 'murder': 1, 'air': 1, '...
24998    {'good': 1, 'plot': 1, 'everything': 1, 'mind'...
24999     {'bad': 4, 'teen': 2, 'wolf': 1, 'sarcastic': 1}
25000                                                   {}
Length: 25001, dtype: object

In [27]:
tf = pd.DataFrame(list(BOW[:-1]))

In [28]:
tf = tf.fillna(0)
df = (tf > 0).sum(axis=0)
idf = np.log(len(tf) / df)
tf_idf_test = tf * idf
#np.save("idf.npy",idf)
# AFTER CAN JUMP TO TEST

In [29]:
ratings = np.array([int(i) for i in ratings[:-1]])
ratings = pd.Series((2*(ratings > 5).astype(int))-1)
tf_idf_test["Label"] =  ratings

In [30]:
tf_idf_test.head()

,good,well,comedy,overcome,coaxed,also,love,something,must,tv,...,aggrandizement,uneventfully,feng,satans,day-for-night,preppie,dines,bleibtreu,terminating,Label
0,0.997067,2.532548,2.375586,5.330841,8.740337,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.997067,1.266274,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.000000,3.798821,0.000000,0.000000,0.000000,1.379756,1.735455,1.821889,2.212013,2.49423,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.997067,3.798821,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


## Saving Train and Test DataFrames

In [31]:
os.chdir(maindir)

intersection = list(set(tf_idf_train.columns).intersection(set(tf_idf_test.columns)))
tf_idf_train = tf_idf_train[intersection]
tf_idf_test = tf_idf_test[intersection]

tf_idf_train.to_csv("standford_train.csv")
tf_idf_test.to_csv("stanford_test.csv")